In [1]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"herutriana","key":"2c1f797ca47ea0ac874fea831a498631"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d nodoubttome/skin-cancer9-classesisic

 99% 777M/786M [00:21<00:00, 41.4MB/s]
100% 786M/786M [00:21<00:00, 38.7MB/s]


In [5]:
import tensorflow as tensor
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [6]:
import zipfile,os
zip_lcl = '/content/skin-cancer9-classesisic.zip'
rzip = zipfile.ZipFile(zip_lcl, 'r')
rzip.extractall('/tmp')
rzip.close()

In [7]:
dirbase = '/tmp/Skin cancer ISIC The International Skin Imaging Collaboration'
dir_latih = os.path.join(dirbase, 'Train')
dir_valid = os.path.join(dirbase, 'Test')

In [8]:
os.listdir('/tmp/Skin cancer ISIC The International Skin Imaging Collaboration/Train')

['nevus',
 'squamous cell carcinoma',
 'seborrheic keratosis',
 'pigmented benign keratosis',
 'melanoma',
 'basal cell carcinoma',
 'dermatofibroma',
 'vascular lesion',
 'actinic keratosis']

In [9]:
os.listdir('/tmp/Skin cancer ISIC The International Skin Imaging Collaboration/Test')

['nevus',
 'squamous cell carcinoma',
 'seborrheic keratosis',
 'pigmented benign keratosis',
 'melanoma',
 'basal cell carcinoma',
 'dermatofibroma',
 'vascular lesion',
 'actinic keratosis']

In [10]:
dir = os.path.join(dirbase,'rockpaperscissors/rps-cv-images')

In [11]:
latih_generator = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    validation_split=0.8,
    shear_range=0.1,
    fill_mode='nearest')
valid_generator = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.8)

In [12]:
generator_latih = latih_generator.flow_from_directory(
    directory=dir_latih,
    target_size=(150,150),
    batch_size=15,
    subset='training',
    class_mode='categorical')
generator_valid = valid_generator.flow_from_directory(
    directory=dir_valid,
    target_size=(150,150),
    batch_size=15,
    subset='validation',
    class_mode='categorical')

Found 452 images belonging to 9 classes.
Found 88 images belonging to 9 classes.


In [13]:
model = tensor.keras.models.Sequential([
  tensor.keras.applications.EfficientNetB7(weights = 'imagenet', include_top = False, input_shape = (150,150,3),classes=9,classifier_activation='softmax'),
  tensor.keras.layers.Normalization(),
  tensor.keras.layers.Flatten(),
  tensor.keras.layers.Dense(512,activation='softmax'),
  tensor.keras.layers.Dense(9,activation='softmax')
  ])

258088960/258076736 [==============================] - 5s 0us/step


In [14]:
model.compile(loss=tensor.keras.losses.MeanSquaredError(),optimizer=tensor.optimizers.Adam(),
              metrics=['accuracy'])

In [15]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
earstop = EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=10)
cekmodel = ModelCheckpoint(filepath='model.hdf5',monitor='val_accuracy',mode='max',verbose=1,save_best_only=True)

In [ ]:
history = model.fit(
    generator_latih,
    steps_per_epoch=20,
    epochs=110,
    validation_data=generator_valid,
    validation_steps=5,
    verbose=2,callbacks=[cekmodel])

Epoch 1/110

Epoch 1: val_accuracy improved from -inf to 0.13333, saving model to model.hdf5
20/20 - 76s - loss: 0.0979 - accuracy: 0.1638 - val_loss: 0.0985 - val_accuracy: 0.1333 - 76s/epoch - 4s/step


In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Akurasi Model Klasifikasi Kanker Kulit')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss Model Klasifikasi RPS')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
pengubah = tensor.lite.TFLiteConverter.from_keras_model(model)
tflite_model = pengubah.convert()

In [ ]:
with tensor.io.gfile.GFile('model.tflite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
model.save('model.pb') 
%cd